# QMC Agent - Jupyter Notebook (Subprocess Version)

Esta versión ejecuta Playwright en un **proceso separado** usando subprocess,
lo que elimina completamente cualquier conflicto con el event loop de asyncio.

## Stack Tecnológico
- **Orquestación**: LangGraph
- **Scraping**: Playwright (via subprocess)
- **LLM**: Groq (LLaMA 3.3 70B)
- **Framework LLM**: langchain_groq

## Flujo Simplificado
1. **Login** → Autenticación en QMC
2. **Filter/Extract** → Extrae TODOS los datos de la tabla
3. **Analyze** → LLM filtra por `Last execution = Today` + `Tags = FE_HITOS`

## IMPORTANTE
**Reinicia el kernel antes de ejecutar** para cargar los módulos actualizados.

## 1. Instalación de Dependencias

In [7]:
# Instalar dependencias (ejecutar solo una vez)
%pip install langgraph langchain-groq langchain-core playwright python-dotenv pydantic
!playwright install chromium

Note: you may need to restart the kernel to use updated packages.


## 2. Configuración y Validación

In [1]:
# Cargar configuración
import sys
sys.path.insert(0, '.')

from src.config import Config

# Validar configuración
missing = Config.validate()
if missing:
    print(f"Configuracion incompleta. Faltan: {', '.join(missing)}")
    print("\nPor favor, completa el archivo .env con los valores requeridos.")
else:
    print("Configuracion valida!")
    print(f"   URL: {Config.QMC_URL}")
    print(f"   Usuario: {Config.QMC_USERNAME}")
    print(f"   Modelo LLM: {Config.GROQ_MODEL}")
    print(f"   Headless: {Config.HEADLESS}")

Configuracion valida!
   URL: https://apqs.grupoefe.pe/qmc/tasks
   Usuario: srvlimprdap011\consultorqs1
   Modelo LLM: llama-3.3-70b-versatile
   Headless: True


## 3. Crear Estado Inicial

In [2]:
from src.state import create_initial_state

# Crear estado inicial
state = create_initial_state()
print("Estado inicial:")
print(f"  current_step: {state['current_step']}")
print(f"  retry_count: {state['retry_count']}")

Estado inicial:
  current_step: init
  retry_count: 0


## 4. Paso 1: Login

In [3]:
from src.nodes.login_node_sync import login_node_sync

print("Ejecutando login (subprocess)...")
print("Esto puede tomar unos segundos...")
login_result = login_node_sync(state)

print(f"\nResultado:")
print(f"  success: {login_result.get('success')}")
print(f"  current_step: {login_result.get('current_step')}")
print(f"  error: {login_result.get('error_message')}")

if login_result.get('session_cookies'):
    print(f"  cookies: {len(login_result.get('session_cookies'))} cookies guardadas")

# Actualizar estado
state.update(login_result)

# Mostrar logs
for log in login_result.get('logs', []):
    print("\n" + log)

Ejecutando login (subprocess)...
Esto puede tomar unos segundos...

Resultado:
  success: True
  current_step: filter
  error: None
  cookies: 1 cookies guardadas

[2026-01-27T13:56:11.614735] LOGIN_SCRIPT: Starting authentication
  Navigating to https://apqs.grupoefe.pe/qmc/tasks
  Checking if already logged in...
  Not auto-logged, filling credentials...
  Credentials submitted
  Waiting for SPA to load...
  Grid/table loaded!
  Task rows visible!
  Login successful!


## 5. Paso 2: Extraer Datos de la Tabla

In [4]:
from src.nodes.filter_node_sync import filter_node_sync

if state.get('current_step') == 'filter':
    print("Extrayendo datos de la tabla (subprocess)...")
    filter_result = filter_node_sync(state)
    
    print(f"\nResultado:")
    print(f"  success: {filter_result.get('success')}")
    print(f"  current_step: {filter_result.get('current_step')}")
    print(f"  error: {filter_result.get('error_message')}")
    
    if filter_result.get('raw_table_data'):
        import json
        data = json.loads(filter_result['raw_table_data'])
        print(f"  total_rows: {data.get('totalRows', 0)}")
        print(f"  headers: {data.get('headers', [])}")
    
    state.update(filter_result)
    
    for log in filter_result.get('logs', []):
        print("\n" + log)
else:
    print(f"No se puede extraer. Estado actual: {state.get('current_step')}")
    if state.get('error_message'):
        print(f"Error previo: {state.get('error_message')}")

Extrayendo datos de la tabla (subprocess)...

Resultado:
  success: True
  current_step: analyze
  error: None
  total_rows: 8
  headers: ['Name', 'Associated resource', 'Type', 'Enabled', 'Status', 'Last execution', 'Next execution', 'Tags', '', '', 'Process', 'Name', 'Duration', '']

[2026-01-27T13:57:25.567031] Page loaded | Clicked Tags filter | Typed 'FE_HITOS_DIARIO' | Tags filter error: Locator.click: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("text=FE_HITOS_DIARIO").first
    - locator resolved to <div class="qmc-table-tag" title="FE_HITOS_DIARIO">FE_HITOS_DIARIO</div>
  - attempting click action
    2 × waiting for element to be visible, enabled and stable
      - element is visible, enabled and stable
      - scrolling into view if needed
      - done scrolling
      - <div class="content" ng-transclude="">…</div> from <qmc-table-column-filter-popover filter="filter" align-to="alignTo" action-defer="actionDefer" class="ng-scope ng-isolate-scope" table-control

## 6. Preview de Datos Extraídos

In [5]:
import json

if state.get('raw_table_data'):
    data = json.loads(state['raw_table_data'])
    print(f"Total filas extraídas: {data.get('totalRows', 0)}")
    print(f"\nHeaders: {data.get('headers', [])}")
    print("\nPrimeras 5 filas:")
    for i, row in enumerate(data.get('rows', [])[:5]):
        print(f"\n--- Fila {i+1} ---")
        for key, value in row.items():
            print(f"  {key}: {value[:50] if len(str(value)) > 50 else value}")
else:
    print("No hay datos extraídos")

Total filas extraídas: 8

Headers: ['Name', 'Associated resource', 'Type', 'Enabled', 'Status', 'Last execution', 'Next execution', 'Tags', '', '', 'Process', 'Name', 'Duration', '']

Primeras 5 filas:

--- Fila 1 ---
  Name: APP_Base Hitos Actualizado
  Associated resource: Base Hitos Actualizado
  Type: Reload
  Enabled: Yes
  Status: Success
  Last execution: 2026-01-27 10:02
  Next execution: On multiple triggers
  Tags: FE_HITOS_DIARIO

--- Fila 2 ---
  Name: APP_Tablero Eficiencia Comercial Actualizado
  Associated resource: Tablero Eficiencia Comercial Actualizado
  Type: Reload
  Enabled: Yes
  Status: Success
  Last execution: 2026-01-27 10:02
  Next execution: On multiple triggers
  Tags: FE_HITOS_DIARIO

--- Fila 3 ---
  Name: CARGA_FACT_HITOS_BT_EXTRACCION
  Associated resource: CARGA_FACT_HITOS_BT_EXTRACCION
  Type: Reload
  Enabled: No
  Status: Skipped
  Last execution: 2026-01-27 08:41
  Next execution: On task event trigger
  Tags: FE_HITOS_DIARIO

--- Fila 4 ---
  Nam

## 7. Paso 3: Analizar con LLM

In [ ]:
from src.analyst import analyst_node_sync
import json

if state.get('current_step') == 'analyze':
    print("\nEsto puede tomar unos segundos...")
    
    analyst_result = analyst_node_sync(state)
    
    print(f"\nResultado:")
    print(f"  current_step: {analyst_result.get('current_step')}")
    print(f"  error: {analyst_result.get('error_message')}")
    
    state.update(analyst_result)
    
    for log in analyst_result.get('logs', []):
        print("\n" + log)
    
    if analyst_result.get('structured_data'):
        print(f"\n\n=== TAREAS FILTRADAS ({len(analyst_result['structured_data'])} resultados) ===")
        print(json.dumps(analyst_result['structured_data'], indent=2, ensure_ascii=False))
else:
    print(f"No se puede analizar. Estado actual: {state.get('current_step')}")
    if state.get('error_message'):
        print(f"Error: {state.get('error_message')}")

Procesando con LLM (Groq)...
El LLM filtrará por: Last_execution=TODAY y Tags=FE_HITOS

Esto puede tomar unos segundos...

Resultado:
  current_step: done
  error: None

[2026-01-27T13:57:27.914660] ANALYST: Analyzing process status
  Processing 8 tasks
  Sample task keys: ['Name', 'Associated resource', 'Type', 'Enabled', 'Status', 'Last execution', 'Next execution', 'Tags']
  Unique statuses found: {'Skipped', 'Success'}
  Process: FE_HITOS_DIARIO
  Counts: 7 completadas, 1 saltadas, 0 ejecutando, 0 pendientes, 0 fallidas
  Estado final: Completado


=== TAREAS FILTRADAS (8 resultados) ===
[
  {
    "Name": "INICIO_MALLA_NZ_HITOS_BT",
    "Associated resource": "INICIO_MALLA_NZ_HITOS_BT",
    "Type": "Reload",
    "Enabled": "Yes",
    "Status": "Success",
    "Last execution": "2026-01-27 08:24",
    "Next execution": "2026-01-28 06:15",
    "Tags": "FE_HITOS_DIARIO"
  },
  {
    "Name": "CARGA_FACT_HITOS_BT_EXTRACCION",
    "Associated resource": "CARGA_FACT_HITOS_BT_EXTRACCION",
 

## 8. Exportar Resultados

In [23]:
import json
from datetime import datetime

if state.get('structured_data'):
    output_file = f"qmc_tasks_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump({
            "extracted_at": datetime.now().isoformat(),
            "filter_criteria": {
                "last_execution": "Today",
                "tags": "FE_HITOS_DIARIO"
            },
            "total_tasks": len(state['structured_data']),
            "tasks": state['structured_data']
        }, f, indent=2, ensure_ascii=False)
    
    print(f"Resultados exportados a: {output_file}")
    print(f"Total tareas: {len(state['structured_data'])}")
else:
    print("No hay datos para exportar.")
    print(f"Estado actual: {state.get('current_step')}")
    if state.get('error_message'):
        print(f"Ultimo error: {state.get('error_message')}")

Resultados exportados a: qmc_tasks_20260127_123124.json
Total tareas: 8
